In [ ]:
import csv
import jgraph as ig
import json
import plotly
# if you want draw offline plot with plotly
import plotly.offline as py
import plotly.graph_objs as go
# if you want draw online plot with plotly
# import plotly.plotly as py
# plotly.tools.set_credentials_file(username='********', api_key='**********')

#### Structure of dict

In [ ]:
# c={'nodes':[{'name':},{'name':}], <br>
#     'links': [{'source': 1, 'target': 0, 'value': 1}]}  <br>

<font size="4">
for more details see </font>
<a href='https://plot.ly/python/3d-network-graph/'>ref</a>

In [ ]:
def find_last(row, name):
    # going from end to start and find first coincidence with 'name'
    for a in row[::-1]:
        if a['name'] == name:
            return a['id']
        else:
            res = None
    return res


def row_transform(row, c_dict):
    # transformation human reliability notation to dict with 'nodes' and 'links'
    if c_dict == {}:
        c_ = 0
        # --------------nodes-----------------------------
        c_dict['nodes'] = [{'name': i, 'id': c_, 'group': 1, 'size': 6}
                           for c_, i in enumerate(row)]
        # --------------links-----------------------------
        c_dict['links'] = [{'source': 0, 'target': i, 'value': 1,
                            'source_name': row[0], 'target_name':row[i]} for i in range(1, len(row))]
    else:
        for i in row[1::]:
            # --------------nodes-----------------------------
            c_ = len(c_dict['nodes'])
            c_dict['nodes'].append(
                {'name': i, 'id': c_, 'group': 1, 'size': 6})
            # ---------links----------------------------------
            res = find_last(c_dict['nodes'], row[0])
            if res != None:
                c_dict['links'].append({'source':
                                        res,
                                        'target': c_, 'value': 1,
                                       'source_name': c_dict['nodes'][res]['name'],
                                        'target_name': c_dict['nodes'][c_]['name']})
            c_ += 1
    return c_dict


def preprocessing_(row, fname):
    # creating  parent,child1_parent,child2_parent,... notation to avoid overlap due to equation parents name and child name
    with open(fname, 'a') as f:
        t_ = row[0]+','+','.join([i+'_'+row[0].split('_')[0]
                                 for i in row[1::]])+'\n'
        f.write(t_)


def check_collisions_(c_dict):
    # collisions breaks up tree? check this
    k = 1
    check = {}
    for i in c_dict['links'][::-1]:
        k += 1
        check[i['source_name']] = 'fail'
        for j in c_dict['links'][::-1]:
            if i['source_name'] == j['target_name']:
                check[i['source_name']] = 'ok'
        if check[i['source_name']] == 'fail':
            print('gaps_warning:', i['source_name'])
    return check


def check_repetition_name_surname(fname):
    # find repetition  pairs parent name and children names to mask this
    a = []
    with open(fname, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            preprocessing_(row, fname_exp)
            a.append(row[0])
    d = {x: a.count(x) for x in a if a.count(x) > 1}
    print('problem_nodes', d)


def beacon(c_dict, tar_):
    # find main line from Adam to Jesus Christ and select it
    c_dict['nodes'][0]['size'] = 10
    c_dict['nodes'][0]['group'] = 2
    for i in range(len(c_dict['nodes'])-1, 1, -1):
        c_dict['nodes'][i]['size'] = 6
        if c_dict['nodes'][i]['name'] == tar_:
            c_dict['nodes'][i]['group'] = 2
            c_dict['nodes'][i]['size'] = 10
            print(c_dict['nodes'][i]['group'])
            for j in c_dict['links'][::-1]:
                if j['target_name'] == tar_:
                    tar_ = j['source_name']
                    print(tar_)
                    break

    return c_dict


#### Start from here

In [ ]:
f = open('tree_explaned.txt', 'w')  # clear up file
f.close()
fname_exp = 'tree_explaned.txt'
fname = 'Source.txt'

In [ ]:
check_repetition_name_surname(fname)

#### Create dict with nodes and links keys

In [ ]:
with open(fname_exp, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    c_dict = {}
    k_ = 0
    for row in spamreader:
        c_dict = row_transform(row, c_dict)

In [ ]:
c_dict['links']

#### Write to file, to make a possible to check data manually

In [ ]:
with open('dict.txt', 'w') as f:
    f.write(json.dumps(c_dict, ensure_ascii=False))

#### Check gaps, gaps breaks up three

In [ ]:
chk_ = check_collisions_(c_dict)

In [ ]:
c_dict

#### Beacon the main line

In [ ]:
c_ = beacon(c_dict, 'Иисус Христос_Мария')
c_

In [ ]:
data = c_

#### Using jgraph to generate coordinates

In [ ]:
N = len(data['nodes'])

L = len(data['links'])
Edges = [(data['links'][k]['source'], data['links'][k]['target'])
         for k in range(L)]

layt = ig.generate(Edges, iterations=100)  # to debug use small iteration

#### Drawing the graph with jgraph

In [ ]:
G = ig.draw(Edges, directed=False)

#### Padding acquired coordinates to scatter 3d plot in plotly

In [ ]:
labels = []
group = []
size_ = []

Xn = []
Yn = []
Zn = []


In [ ]:
for k, node in enumerate(data['nodes']):
    labels.append(node['name'].split('_')[0].split('(')[0])
    group.append(node['group'])
    size_.append(node['size'])

    Xn.append(layt['nodes'][k]['location'][0])  # x-coordinates of nodes
    Yn.append(layt['nodes'][k]['location'][1])  # y-coordinates
    Zn.append(layt['nodes'][k]['location'][2])  # z-coordinates


In [ ]:
Xe = []
Ye = []
Ze = []
for e in Edges:
    Xe += [layt['nodes'][e[0]]['location'][0], layt['nodes']
           [e[1]]['location'][0], None]  # x-coordinates of edge ends
    Ye += [layt['nodes'][e[0]]['location'][1],
           layt['nodes'][e[1]]['location'][1], None]
    Ze += [layt['nodes'][e[0]]['location'][2],
           layt['nodes'][e[1]]['location'][2], None]


In [ ]:
trace1 = go.Scatter3d(x=Xe,
                      y=Ye,
                      z=Ze,
                      mode='lines',
                      line=dict(color='rgb(125,125,125)', width=1),
                      hoverinfo='none'
                      )

trace2 = go.Scatter3d(x=Xn,
                      y=Yn,
                      z=Zn,
                      mode='markers+text',
                      name='actors',
                      marker=dict(symbol='circle',
                                  size=size_,
                                  color=group,
                                  colorscale='Viridis',
                                  line=dict(color='rgb(50,50,50)', width=0.5)
                                  ),
                      text=labels,
                      hoverinfo='text'
                      )

axis = dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title=''
            )

layout = go.Layout(
    title="Генеалогическое древо от Адама до Иисуса Христа",
    width=1000,
    height=1000,
    showlegend=False,
    scene=dict(
        xaxis=dict(axis),
        yaxis=dict(axis),
        zaxis=dict(axis),
    ),
    margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
        dict(
            showarrow=False,
            text="Data source: <a href='https://azbyka.ru/shemy/genealogicheskoe-drevo-ot-adama-i-evy-do-iisusa-khrista.shtml'>ref</a>",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
                size=14
            )
        )
    ],)

data = [trace1, trace2]


#### Drawing plot with plotly

In [ ]:
# plot to file
# fig=go.FigureWidget(data=data, layout=layout)
# py.plot(fig, filename='Adam.html')

In [ ]:
# plot online
# fig=go.Figure(data=data, layout=layout)
# py.iplot(fig, filename='Adam_tree.html')

In [ ]:
# plot in notebook
fig = go.FigureWidget(data=data, layout=layout)
display(fig)